In [369]:
import joblib
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import csr_matrix

[nltk_data] Downloading package punkt to C:\Users\zxc
[nltk_data]     ghoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import faiss

In [3]:
import torch
from transformers import BertTokenizer, BertModel

In [392]:
db_name = 'new.db'
conn = sqlite3.connect(db_name)

#забираем доступные категории
ingredients_sql = f"select i.id, i.name_ingr from ingredient i"

loaded_categories = pd.read_sql(ingredients_sql, conn)

ids_ingr = []
for i in range(len(loaded_categories)):
    ids_ingr.append(str(loaded_categories.id[i]))

#print(ids_ingr)

#забираем способ приготовления рецепта с категориями из доступных с id категорий
sql = f"SELECT r.id, r.manual, (SELECT group_concat(ri.ingr_id , ', ') " \
      f"from recipe_ingredients ri " \
      f"WHERE ri.recipe_id = r.id AND " \
      f"ri.ingr_id in ({', '.join(ids_ingr)})) " \
      f"AS ingredients from recipe r " \
      f"WHERE LENGTH(r.manual) > 50"

loaded_data = pd.read_sql(sql, conn)

#pandas data frame
conn.close()

In [393]:
loaded_data

,id,manual,ingredients
0,1,Шоколад разломать на кусочки и вместе со сливо...,"1, 2, 3, 4, 5, 6"
1,2,Положите весь творог в кастрюльку и разомните ...,"4, 5, 7, 8, 9"
2,3,Вскипятите воду в большой кастрюле и сварите п...,"10, 11, 12, 13, 14, 15, 16, 17, 2, 4"
3,4,Разогреть духовку. Отделить белки от желтков. ...,"18, 19, 20, 4, 5, 8"
4,5,Взбить яйца с сахаром.\n\nПостепенно ввести му...,"16, 19, 21, 4, 5, 8"
...,...,...,...
19039,54988,"Отварную куриную грудку нарезаем соломкой, вык...","102, 2795, 2870, 2969, 904"
19040,54990,Отделить белок от желтка.\r\nБелок поместить в...,"21, 2780, 2800, 2814, 2826, 8"
19041,54991,30 граммов имбиря почистить и нарезать.Измельч...,"2853, 3113, 621, 67, 8, 84"
19042,54992,"Беру 5 яиц, добавляю 2 ст. л. сахара с верхом,...","133, 16, 21, 2780, 2786, 2788, 40, 8"


In [394]:
for i in range(len(ids_ingr)):
    val = []
    for j in range(len(loaded_data)):
        spl = str(loaded_data.ingredients[j]).split(', ')
        val.append(int(str(ids_ingr[i]) in spl))
    loaded_data.insert(loc=len(loaded_data.columns) , column=ids_ingr[i], value=val)

C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_10152\3739734041.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loaded_data.insert(loc=len(loaded_data.columns) , column=ids_ingr[i], value=val)
C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_10152\3739734041.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loaded_data.insert(loc=len(loaded_data.columns) , column=ids_ingr[i], value=val)
C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_10152\3739734041.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [395]:
#делим dataset на тренировочный и тестовый
X_train,X_test,y_train,y_test = train_test_split(loaded_data["manual"], loaded_data[ids_ingr],test_size=0.3,random_state=42)

In [406]:
class BertVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny')
        self.model = BertModel.from_pretrained('cointegrated/rubert-tiny')
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectors = []
        for text in X:
            text_vector = self.__vectorize__(text, model=self.model, tokenizer=self.tokenizer)
            vectors.append(text_vector)

        # Преобразование списка в разреженную матрицу
        X_sparse = csr_matrix(vectors)

        return X_sparse

    def __vectorize__(self, text, model, tokenizer):
        t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()

In [407]:
class FaissKNeighbors:
    def __init__(self):
        self.index = None
        self.y = None
        self.vectorizer = BertVectorizer()
        self.normalizer = Normalizer()
        self.knn_neighbors = None  # Добавленный атрибут
        self.text_mapping = {}  # Маппинг между векторами и текстовыми описаниями

    def fit(self, X, y):
        X_vectors = self.vectorizer.transform(X)
        X_dense = X_vectors.toarray().astype(np.float32)

        X_normalized = self.normalizer.fit_transform(X_dense)

        dimension = X_normalized.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(X_normalized)

        self.y = np.asarray(y)  # Преобразование меток в массив numpy

        # Сохранение маппинга между векторами и текстовыми описаниями
        for i, text in enumerate(X):
            self.text_mapping[i] = text

        return self

    def predict(self, recipe, k=5):
        recipe_vector = self.vectorizer.transform([recipe])
        recipe_vector = self.normalizer.transform(recipe_vector)
        recipe_dense = recipe_vector.toarray().astype(np.float32)

        distances, indices = self.index.search(recipe_dense, k)

        knn_labels = self.y[indices]

        self.knn_neighbors = knn_labels.tolist()  # Сохранение ближайших соседей в виде списка

        #Преобразование векторов обратно в текст
        knn_texts = [self.text_mapping[i] for i in indices[0]]

        return knn_texts

    def get_top_recipes(self, recipe_list, k=5):
        # Получаем предсказанных соседей для каждого рецепта в списке
        neighbors_list = [self.predict(recipe) for recipe in recipe_list]

        # Считаем количество вхождений каждого рецепта в список соседей
        recipe_counts = {}
        for neighbors in neighbors_list:
            for recipe in neighbors:
                if recipe in recipe_counts:
                    recipe_counts[recipe] += 1
                else:
                    recipe_counts[recipe] = 1

        # Сортируем рецепты по количеству вхождений в общий список соседей
        sorted_recipes = sorted(recipe_counts.items(), key=lambda x: x[1], reverse=True)

        # Выбираем топ-K наиболее подходящих рецептов
        top_recipes = [recipe for recipe, _ in sorted_recipes[:k]]

        return top_recipes

In [408]:
knn_faiss = FaissKNeighbors()

knn_faiss.fit(X_train, y_train)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [154]:
X_test[8]

'Яйца взбить с сахаром до белой пены. Добавить 2 стакана просеянной муки, соду, растопленное остывшее сливочное масло, сметану и тщательно перемешать (лучше миксером).\n\nТесто разделить на две равные части. В одну часть добавить 2 столовые ложки муки, в другую 2 столовые ложки какао. Перемешать, чтобы не было комочков. Тесто должно быть консистенции негустой сметаны.\n\nШирокую форму (26–28 см), смазать маслом. Вливать в центр поочередно небольшие порции разного теста (по столовой ложке). Не перемешивать.\n\nВыпекать в предварительно разогретой духовке при температуре 200 градусов в течение получаса. Если верх пирога уже пропечется, а середина еще нет — следует накрыть пирог фольгой, уменьшить температуру до 180 градусов и выпекать до готовности.\n\n'

In [409]:
recipe = X_test[8]
print(recipe)
similar_neighbors = knn_faiss.predict(recipe, k = 30)

Яйца взбить с сахаром до белой пены. Добавить 2 стакана просеянной муки, соду, растопленное остывшее сливочное масло, сметану и тщательно перемешать (лучше миксером).

Тесто разделить на две равные части. В одну часть добавить 2 столовые ложки муки, в другую 2 столовые ложки какао. Перемешать, чтобы не было комочков. Тесто должно быть консистенции негустой сметаны.

Широкую форму (26–28 см), смазать маслом. Вливать в центр поочередно небольшие порции разного теста (по столовой ложке). Не перемешивать.

Выпекать в предварительно разогретой духовке при температуре 200 градусов в течение получаса. Если верх пирога уже пропечется, а середина еще нет — следует накрыть пирог фольгой, уменьшить температуру до 180 градусов и выпекать до готовности.




In [414]:
recipes = X_test[5:68].tolist()
similar_neighbors = knn_faiss.get_top_recipes(recipes)

In [415]:
recipes

['Духовку нагреть до 170-180 С.\n\r\nИз вишни (черешни) достать косточки. Если вишня будет замороженная – не размораживать.\r\nВзбить масло с сахаром и ванилином до пышности – минут 10.\r\nДобавить творог, смешать. Добавить яйца по одному. Просеять муку с разрыхлителем, содой, солью, крахмалом – добавить ее в тесто. Взбивать минут 5.\r\nМасло и сахар сбивать 15 мин.миксером на средней скорости. Добавить творог, сбить до однородности. Добавить яйца, соду, разрыхлитель, ванильный сахар, соль и перемешать.  Всыпать муку и замесить тесто в течение 5мин.\r\nФорму взяла силиконовую в виде розы. Имея печальный опыт застревания в ней кекса, все же хорошо смазала ее маслом. Можно печь в кексовых мелких формочках.\r\nВыложила тесто в форму, сверху разложила, чуть вдавив, ягоды и отправила все это выпекаться. \nПеклось долго, около часа. Проверяла зубочисткой на готовность.\r\nПирогу обязательно дать остыть в форме полчаса, потом перевернуть и дать еще постоять с полчаса. Верх посыпать пудрой или

In [416]:
similar_neighbors

['На чистую рабочую поверхность горкой просеять муку и крахмал, сделать небольшое углубление посередине. В это углубление добавить соль, сахар и влить воду с маслом.\n\nОсторожно поднять муку с краев в середину и замесить крутое тесто. Вымешивать его нужно долго, 10–12 минут, пока тесто не превратится в упругий, не очень липкий шар.\n\nНакрыть тесто пленкой или влажным полотенцем и оставить на 30 минут при комнатной температуре. Пока тесто отдыхает, можно заняться начинкой.\n\nДля начинки отварить куриные бедра, снять кожу, отделить мясо и пропустить его через мясорубку. Лук нарезать кубиками, обжарить на растительном масле, добавить грибы, также нарезанные кубиками, и обжарить все вместе. К фаршу добавить жареные грибы и лук, соль, перец, влить куриный бульон и тщательно перемешать.\n\nНа припыленной мукой рабочей поверхности раскатать тесто в тонкий пласт, вырезать с помощью формы кружочки. На середину каждого кружочка выложить начинку, защипнуть края, придать форму вареника.\n\nВаре

In [1]:
joblib.dump(knn_faiss, "./knn.joblib")

NameError: name 'joblib' is not defined